## AlphaNeo for Max Length of 3 (depth of 4)

```
tensorboard --logdir runs
```

```
cd ./Trinity/
python ./AlphaNeo_pworker.py 1
```

```
nohup jupyter lab > jupyter.log &
```

In [1]:
DBG_VAR = None
DBG_SEXP = None
DBG_POST = None

In [2]:
import logging 
logging.basicConfig(level=logging.CRITICAL)

In [3]:
import os
import itertools
import copy
import random
import fcntl

from pathlib import Path

In [4]:
import tyrell.spec as S
from tyrell.interpreter import Interpreter, PostOrderInterpreter, GeneralError, InterpreterError
from tyrell.enumerator import Enumerator, SmtEnumerator, RandomEnumerator, DesignatedEnumerator, RandomEnumeratorS, ExhaustiveEnumerator
from tyrell.decider import Example, ExampleConstraintPruningDecider, ExampleDecider, TestDecider
from tyrell.synthesizer import Synthesizer
from tyrell.logger import get_logger
from sexpdata import Symbol
from tyrell import dsl as D
from typing import Callable, NamedTuple, List, Any

In [5]:
# import pickle
import dill as pickle
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from tensorboardX import SummaryWriter

use_cuda = torch.cuda.is_available()
print("use_cuda: {}".format(use_cuda))

use_cuda: False


In [6]:
# Morpheus Version
from utils_morpheus import *
from ProgramSpace import *

In [7]:
torch.__version__

'1.1.0'

In [8]:
class ListModule(object):
    def __init__(self, module, prefix, *args):
        self.module = module
        self.prefix = prefix
        self.num_module = 0
        for new_module in args:
            self.append(new_module)
    
    def append(self, new_module):
        if not isinstance(new_module, nn.Module):
            raise ValueError('Not a Module')
        else:
            self.module.add_module(self.prefix + str(self.num_module), new_module)
            self.num_module += 1
            
    def __len__(self):
        return self.num_module
    
    def __getitem__(self, i):
        if i<0 or i>=self.num_module:
            raise IndexError('Out of bound')
        return getattr(self.module, self.prefix+str(i))

In [9]:
class AlphaNeo(nn.Module):
    def __init__(self, p_config=None):
        super(AlphaNeo, self).__init__()
        self.config = p_config
        
        self.lstm = nn.LSTMCell(
            input_size = self.config["lstm_input_size"],
            hidden_size = self.config["fcs"][0],
            bias = True,
        )
        
        self.fcs = ListModule(self, "fc_")
        for i in range(1,len(self.config["fcs"])):
            self.fcs.append(
                nn.Linear(self.config["fcs"][i-1], self.config["fcs"][i])
            )
    
    '''
    single batch behavior, no batch dim expected
    '''
    def forward(self, p1, p2, p3, hc=None):
        # p1, p2: (1, ?)
        # hc: previous hidden state, if None, start a new sequence
        
        # [(1, ?), (1, ?), ...] -> (1, ?*k), flatten
        d_known = torch.cat([p1,p2,p3], dim=1)
        
        if hc is None:
            hc = self.init_hidden()
        
        h_t, c_t = self.lstm(d_known, hc)
        
        tmp1 = h_t
        for i in range(len(self.fcs)-1):
            tmp1 = F.relu(self.fcs[i](tmp1))
        
        # (1, #production_rules)
        tmp1 = F.log_softmax(self.fcs[len(self.fcs)-1](tmp1), dim=1)
        
        # hidden states can be reused
        return tmp1, (h_t,c_t)
    
    def init_hidden(self):
        if use_cuda:
            return (torch.zeros(1,self.config["fcs"][0]).cuda(),
                    torch.zeros(1,self.config["fcs"][0]).cuda())
        else:
            return (torch.zeros(1,self.config["fcs"][0]),
                    torch.zeros(1,self.config["fcs"][0]))

In [10]:
def AlphaNeoTrainer(p_config, p_spec, p_interpreter, p_generator, p_model, p_lossfn, p_optim, p_writer):
    global DBG_VAR, DBG_SEXP, DBG_POST
    loss_list = []
    n_batch = 32
    batch_loss = 0.
    c_nth = 0
    
    # #### mp #### #
    for ii in [1,2,3]:
        data_path = "./pworker_storage/data_{}.pkl".format(ii)
        Path(data_path).touch()
    external_data = []
    # iterator
    iter_external_data = iter(external_data)
    need_reload = True
    # #### mp #### #
    
    total_ac = {1:0,2:0,3:0}
    total_sp = {1:0,2:0,3:0}
    # one program each step
    for d_step in range(p_config["n_steps"]):
        p_model.train()
        
        # #### mp #### #
        if need_reload:
            for ii in [1,2,3]:
                data_path = "./pworker_storage/data_{}.pkl".format(ii)
                # compete once and get some results
                f = open(data_path,"rb+")
                try:
                    fcntl.flock(f.fileno(), fcntl.LOCK_EX)
                    if os.path.getsize(data_path)>0:
                        external_data = pickle.load(f)
                        if len(external_data)>0:
                            # actually it has data
                            f.seek(0)
                            pickle.dump([],f)
                            iter_external_data = iter(external_data)
                            need_reload = False
                except IOError as e:
                    if e.errno != errno.EAGAIN:
                        # unrelated errors
                        raise
                    else:
                        # fail once, move on generate by itself
                        pass
                f.close()
                if not need_reload:
                    break
        # #### mp #### #
        
        try:
            p_prog, str_example = next(iter_external_data)
            # Morpheus ONLY: convert str_example to p_example
            # assign new names
            p_example = Example(
                input=[get_fresh_name() for p in str_example.input],
                output=get_fresh_name(),
            )
            # load the value into new names in R environment
            for i in range(len(p_example.input)):
                p_interpreter.load_data_into_var(
                    str_example.input[i], # data
                    p_example.input[i], # var name
                )
            p_interpreter.load_data_into_var(
                str_example.output,
                p_example.output,
            )
        except StopIteration:
            need_reload = True
            # run out of sample, generate one 
            p_input = p_interpreter.random_table()
            while True:
                p_prog, p_example = p_generator.generate(
                    fixed_depth=p_config["max_depth"],
                    example=Example(input=[p_input], output=None),
                )
                # make sure at least one function call
                if p_prog.is_apply():
                    break
                
        # construct the full program
        ps_full = ProgramSpaceChainOneNB(
            p_spec, p_interpreter, eq_r, p_example.input, p_example.output,
        )
        p_prog_list = ps_full.get_prog_list(p_prog)
        for p in p_prog_list:
            ps_full.add_sexp(p.to_sexp())
        total_sp[len(ps_full.prog_list)] += 1
        # start from the first state
        ps_current = ProgramSpaceChainOneNB(
            p_spec, p_interpreter, eq_r, p_example.input, p_example.output,
        )
        
        d_loss = 0.

        # roll till the ending condition
        hc_t = None
        for _ in range(len(ps_full.prog_list)):
            outv_current = ps_current.get_frontier()
            # generate dense rep for known nodes
            d_vertex = morpheus_perspective1(outv_current[0])
            # generate dense rep for target
            d_target = morpheus_perspective1(ps_current.output)
            
            # compute the difference
            d_distance = [d_target[d_pos]-d_vertex[d_pos] for d_pos in range(len(d_vertex))]

            if use_cuda:
                td_vertex = Variable(torch.FloatTensor( [d_vertex] )).cuda()
                td_target = Variable(torch.FloatTensor( [d_target] )).cuda()
                td_distance = Variable(torch.FloatTensor( [d_distance] )).cuda()
            else:
                td_vertex = Variable(torch.FloatTensor( [d_vertex] ))
                td_target = Variable(torch.FloatTensor( [d_target] ))
                td_distance = Variable(torch.FloatTensor( [d_distance] ))
                                     
            # (1, LIST_PAD_LENGTH+#production_rules)
            td_output, hc_t = p_model(td_vertex, td_target, td_distance, hc_t)
            cnd_list = ps_full.shell_list

            DBG_VAR = [ps_full, ps_current]
            selected_id = ps_full.str_shell_dict[
                str(ps_full.prog_list[len(ps_current.prog_list)])
            ]
            
            d_loss += p_lossfn(F.log_softmax(td_output,dim=1), torch.tensor([selected_id]))
            
            # add selected edges and fill
            ret = ps_current.add_sexp(ps_current.shell_list[selected_id].to_sexp())
            # debug info
            DBG_SEXP = ps_current.shell_list[selected_id].to_sexp()
            DBG_POST = ps_current
            
            if ret==False:
                raise Exception("This can't happen.")

        loss_list.append(d_loss)
        batch_loss += d_loss

        print("\r# STEP{}, from:{}, size:f{}, loss:{:.4f}, avg.loss:{:.4f}".format(
            d_step, "gen" if need_reload else "load",
            len(ps_full.prog_list), d_loss, sum(loss_list)/len(loss_list),
        ), end="")

        if writer is not None:
            writer.add_scalar(
                'avg.loss/step',
                sum(loss_list)/len(loss_list),
                d_step,
            )
            writer.add_scalar(
                'loss/step',
                d_loss,
                d_step,
            )
        
        
        c_nth += 1
        if c_nth%n_batch==0:
            c_nth = 0
            # perform gradient in every batch
            batch_loss.backward()
            p_optim.step()
            p_optim.zero_grad()
            batch_loss = 0.
            
        if d_step%10000==0:
            torch.save(p_model.state_dict(), "./saved_models/Supervised_AlphaNeo_Morpheus_n3.pt".format(d_step))


In [11]:
m_interpreter = MorpheusInterpreter()
m_spec = S.parse_file('./example/mChainOneNB.tyrell')
m_eq = eq_r
m_generator = MorpheusGenerator(
    spec=m_spec,
    interpreter=m_interpreter,
    sfn=m_interpreter.sanity_check,
)
m_ps = ProgramSpaceChainOneNB(
    m_spec, m_interpreter, m_eq, None, None,
)
# m_config = {
#     'n_steps': 1000000,
#     'gamma': 0.618,
#     'exploration_rate': lambda x:0.9-0.8*(min(1, x/10000)),
#     'hint_rate': lambda x,n:{1:1,2:0.5,3:0.8}[n]*(1-(min(0.7, x/10000))),
#     'max_depth': 4,
#     'gen1_length': 27,
# }
m_config = {
    'n_steps': 1000000,
    'gamma': 0.618,
    'exploration_rate': lambda x:0.9-0.8*(min(1, x/10000)),
    'hint_rate': lambda x,n:{1:0.8,2:0.8,3:0.8}[n]*(1-(min(0.8, x/10000))),
    'max_depth': 4,
    'gen1_length': 63,
}
m_config['lstm_input_size'] = 3*m_config["gen1_length"]
m_config["fcs"] = [
    128,
    len(m_ps.shell_list),
]

alpha_neo = AlphaNeo(p_config=m_config)
optimizer = torch.optim.Adam(list(alpha_neo.parameters()))
lossfn = nn.NLLLoss()
writer = SummaryWriter("runs/Supervised_AlphaNeo_Morpheus_n3")
# writer = None

In [12]:
len(m_ps.shell_list)

390

In [ ]:
AlphaNeoTrainer(m_config, m_spec, m_interpreter, m_generator, alpha_neo, lossfn, optimizer, writer)

# STEP7071, from:load, size:f3, loss:13.8244, avg.loss:15.2084

In [ ]:
DBG_POST.outv_list[-1][0]

In [ ]:
print(robjects.r(DBG_POST.outv_list[-1][0]))

In [ ]:
print(1)

In [ ]:
ret = robjects.r('ncol({})'.format(DBG_POST.outv_list[-1][0]))
ret[0]

In [ ]:
ret = robjects.r('nrow({})'.format(DBG_POST.outv_list[-1][0]))
ret[0]

In [ ]:
DBG_SEXP

In [ ]:
DBG_VAR[0]

In [ ]:
DBG_VAR[0]

In [ ]:
DBG_VAR[1].out_eq()

In [ ]:
DBG_VAR[0].out_eq()

In [ ]:
DBG_VAR[0].get_frontier()

In [ ]:
print(robjects.r(DBG_VAR[0].get_frontier()[0]))

In [ ]:
print(robjects.r(DBG_VAR[0].output))

In [ ]:
print(robjects.r(DBG_VAR[0].outv_list[-1][0]))

In [ ]:
np_obj = numpy.asarray(robjects.r(DBG_VAR[0].outv_list[-1][0])).T

In [ ]:
np_obj.shape

In [ ]:
len(DBG_VAR[0].prog_list)

In [ ]:
len(DBG_VAR[1].prog_list)

In [ ]:
str(DBG_VAR[0].prog_list[len(DBG_VAR[1].prog_list)])